In [9]:
import sys
import os
import numpy as np
import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from keras.utils import plot_model
from keras import backend as K

import SegDataGenerator as gen

# can't run on GPU
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [10]:
data_path = 'data/dataset_256x256.npz'
model_path = 'model/'
learning_rate = 0.0001
batch_size = 1
img_height = 256
img_width = 256
img_channel = 1
steps_per_epoch=5
epochs=5
validation_step=2

In [11]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2, y_true)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)


In [12]:
def segmentation_loss(y_true, y_pred):
    class_prop = 0.1638 
    weight_per_label = tf.scalar_mul(class_prop, tf.cast(tf.equal(y_true, 0), tf.float32)) + \
                        tf.scalar_mul(1.0 - class_prop, tf.cast(tf.equal(y_true, 1), tf.float32))
    cross_entropy = tf.losses.sigmoid_cross_entropy(y_true, y_pred)
    cross_entropy_weighted = tf.multiply(weight_per_label, cross_entropy)
    cross_entropy_mean = tf.reduce_mean(cross_entropy_weighted)
    return cross_entropy_mean

In [13]:
def contour_loss(y_true, y_pred):
    class_prop = 0.2249
    weight_per_label = tf.scalar_mul(class_prop, tf.cast(tf.equal(y_true, 0), tf.float32)) + \
                        tf.scalar_mul(1.0 - class_prop, tf.cast(tf.equal(y_true, 1), tf.float32))
    cross_entropy = tf.losses.sigmoid_cross_entropy(y_true, y_pred)
    cross_entropy_weighted = tf.multiply(weight_per_label, cross_entropy)
    cross_entropy_mean = tf.reduce_mean(cross_entropy_weighted)
    return cross_entropy_mean

In [14]:
class Multi_UNet():
    def __init__(self, learning_rate, batch_size, img_height, img_width, img_channel):
        self.lr = learning_rate
        self.bs = batch_size
        self.height = img_height
        self.width = img_width
        self.channel = img_channel
        self.model = self.create_model()
    
    def ConvBlock(self, x, num_filter, conv_kernel_size=(3,3), dropout=0.2,
                  pool_size=(2,2), hasPool=True):
        conv = Conv2D(num_filter, conv_kernel_size, activation='elu', 
                      kernel_initializer='he_normal', padding='same') (x)
        conv = Dropout(dropout) (conv)
        conv = Conv2D(num_filter, conv_kernel_size, activation='elu', 
                      kernel_initializer='he_normal', padding='same') (conv)
        if hasPool == True:
            pool = MaxPooling2D(pool_size)(conv)
            return conv, pool
        return conv
    
    def DeConvBlock(self, x, cat, num_filter=16, conv_kernel_size=(3,3), up_size=(2,2), 
                    up_stride=(2,2), dropout=0.2, axis=-1):
        deconv = Conv2DTranspose(num_filter, up_size, strides=up_stride, padding='same') (x)
        deconv = concatenate([deconv, cat], axis=axis)
        conv = Conv2D(num_filter, conv_kernel_size, activation='elu', 
                      kernel_initializer='he_normal', padding='same') (deconv)
        conv = Dropout(dropout) (conv)
        conv = Conv2D(num_filter, conv_kernel_size, activation='elu', 
                    kernel_initializer='he_normal', padding='same') (conv)
        return conv
    
    
    def create_model(self):
        inputs = Input((self.height, self.width, self.channel,))
        
        # down-sampling
        conv1, pool1 = self.ConvBlock(inputs, 16, (3,3), 0.1, (2,2))
        conv2, pool2 = self.ConvBlock(pool1, 32, (3,3), 0.1, (2,2))
        conv3, pool3 = self.ConvBlock(pool2, 64, (3,3), 0.2, (2,2))
        conv4, pool4 = self.ConvBlock(pool3, 128, (3,3), 0.2, (2,2))
        conv5 = self.ConvBlock(pool4, 256, (3,3), 0.3, hasPool=False)
        
        # up-sampling
        conv6_s = self.DeConvBlock(conv5, conv4, 128, (3,3), (2,2), (2,2), 0.2)
        conv7_s = self.DeConvBlock(conv6_s, conv3, 64, (3,3), (2,2), (2,2), 0.2)
        conv8_s = self.DeConvBlock(conv7_s, conv2, 32, (3,3), (2,2), (2,2), 0.1)
        conv9_s = self.DeConvBlock(conv8_s, conv1, 16, (3,3), (2,2), (2,2), 0.1, axis=3)
        outputs_s = Conv2D(1, (1, 1), activation='sigmoid', name='segmentation') (conv9_s)
        
        conv6_c = self.DeConvBlock(conv5, conv4, 128, (3,3), (2,2), (2,2), 0.2)
        conv7_c = self.DeConvBlock(conv6_c, conv3, 64, (3,3), (2,2), (2,2), 0.2)
        conv8_c = self.DeConvBlock(conv7_c, conv2, 32, (3,3), (2,2), (2,2), 0.1)
        conv9_c = self.DeConvBlock(conv8_c, conv1, 16, (3,3), (2,2), (2,2), 0.1, axis=3)
        outputs_c = Conv2D(1, (1, 1), activation='sigmoid', name='contour') (conv9_c)
        
        model = Model(inputs=[inputs], outputs=[outputs_s, outputs_c])
        metrics = {'segmentation': [mean_iou], 'contour': [mean_iou]}
        loss = {'segmentation': segmentation_loss, 'contour': contour_loss}
        adam = optimizers.Adam(lr = self.lr)
        model.compile(optimizer=adam, loss=loss, loss_weights=[1.,1.], metrics=metrics)
        print(model.summary())
        
        return model


In [15]:
def main(args):
    # define model
    multi_unet = Multi_UNet(learning_rate, batch_size, img_height, img_width, img_channel)
    model = multi_unet.model
#     plot_model(model, to_file='model.png')
    print("generate model!")
    
    # define generator
    trainGenerator = gen.SegDataGenerator(validation_split=0.2)
    train_data = trainGenerator.flow_from_directory(data_path, subset='training', batch_size=batch_size,
                                                   class_mode='segmentation', color_mode='grayscale',
                                                   use_contour=True)
    val_data = trainGenerator.flow_from_directory(data_path, subset='validation', batch_size=batch_size,
                                                   class_mode='segmentation', color_mode='grayscale',
                                                   use_contour=True)
    print("data generator!")
    
    # training
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    checkpoint = ModelCheckpoint(model_path+'weight.{epoch:02d}.hdf5', monitor='val_loss',
                                 mode='min', period=1)
    from tensorflow.python.client import device_lib
    print(device_lib.list_local_devices())
    model.fit_generator(train_data, steps_per_epoch=steps_per_epoch, epochs=epochs, 
                        callbacks=[checkpoint], validation_data=val_data, 
                        validation_steps=validation_step, shuffle=True)
    
    

In [16]:
if __name__ == '__main__':
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)
    K.tensorflow_backend.set_session(sess)
    main(sys.argv)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_27 (Conv2D)              (None, 256, 256, 16) 160         input_2[0][0]                    
__________________________________________________________________________________________________
dropout_14 (Dropout)            (None, 256, 256, 16) 0           conv2d_27[0][0]                  
__________________________________________________________________________________________________
conv2d_28 (Conv2D)              (None, 256, 256, 16) 2320        dropout_14[0][0]                 
__________________________________________________________________________________________________
max_poolin

Found 535 images belonging to 2 classes.
Reading from previously loaded data.
Found 134 images belonging to 2 classes.
data generator!
[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2423939131184325911
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 3497590784
locality {
  bus_id: 1
}
incarnation: 18150198801056255111
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0"
]
Epoch 1/5
5/5 [==============================] - 2s 321ms/step - loss: 0.2023 - segmentation_loss: 0.1347 - contour_loss: 0.0676 - segmentation_mean_iou: 8.2817e-05 - contour_mean_iou: 0.0183 - val_loss: 0.1809 - val_segmentation_loss: 0.1482 - val_contour_loss: 0.0327 - val_segmentation_mean_iou: 1.3021e-04 - val_contour_mean_iou: 0.0098
Epoch 2/5
5/5 [==============================] - 0s 64ms/step - loss: 0.2029 - segmentation_loss: 0.1545 - contour_loss: 0.0484 - segmentation_mean_iou: 1.0593e-04 - contour_mean_iou: 0.0062 - val_loss: 0.1793 - v